In [1]:
import pandas as pd
import scipy.stats as sts
import matplotlib.pyplot as plt
import numpy as np
import csv
import time
import json
import ast

In [84]:
full_data = pd.read_csv('processed_data.csv')
full_data = full_data.drop('Unnamed: 0', axis=1)
data1 = pd.read_csv('sentiment_remote_change.csv', names=['0','sentiment_remote_change'])
data1 = data1.drop('0', axis=1)
data1 = data1.drop(0, axis=0).reset_index(drop=True)
data2 = pd.read_csv('sentiment_why_mode.csv', names=['0','sentiment_why_mode'])
data2 = data2.drop('0', axis=1)
data2 = data2.drop(0, axis=0).reset_index(drop=True)
data3 = pd.read_csv('sentiment_prior_change.csv', header=None)
data3 = data3.drop(0, axis=1).reset_index(drop=True)
data3.columns = ['sentiment_prior_change']
data4 = pd.read_csv('sentiment_why_preference.csv', header=None)
data4 = data4.drop(0, axis=1).reset_index(drop=True)
data4.columns = ['sentiment_why_preference']
print(full_data.shape, data1.shape, data2.shape, data3.shape, data4.shape)

(10455, 39) (10455, 1) (10455, 1) (10454, 1) (10454, 1)


In [65]:
# data = full_data.join(data1)
# data = data.join(data2)

In [66]:
def to_list(row):
    if not pd.isna(row):
        return ast.literal_eval(row)
    else:
        return np.NaN

In [67]:
def num_sentiment(row):
    if type(row) == list:
        return len(row)
    else:
        return np.NaN

In [100]:
print(data1['sentiment_remote_change'].apply(to_list).apply(num_sentiment).value_counts(normalize=True))
print(data2['sentiment_why_mode'].apply(to_list).apply(num_sentiment).value_counts(normalize=True))
print(data3['sentiment_prior_change'].apply(to_list).apply(num_sentiment).value_counts(normalize=True))
print(data4['sentiment_why_preference'].apply(to_list).apply(num_sentiment).value_counts(normalize=True))

1.0    0.387741
2.0    0.276158
0.0    0.254054
3.0    0.081660
4.0    0.000290
6.0    0.000097
Name: sentiment_remote_change, dtype: float64
1.0    0.421295
2.0    0.300772
0.0    0.196226
3.0    0.081600
4.0    0.000107
Name: sentiment_why_mode, dtype: float64
1.0    0.397833
0.0    0.269350
2.0    0.254714
3.0    0.077259
4.0    0.000704
5.0    0.000141
Name: sentiment_prior_change, dtype: float64
2.0    0.356848
1.0    0.353099
0.0    0.159443
3.0    0.128015
4.0    0.002499
5.0    0.000096
Name: sentiment_why_preference, dtype: float64


In [99]:
data1['sentiment_remote_change'].apply(to_list).apply(num_sentiment).value_counts(normalize=True)

1.0    0.387741
2.0    0.276158
0.0    0.254054
3.0    0.081660
4.0    0.000290
6.0    0.000097
Name: sentiment_remote_change, dtype: float64

In [68]:
# data['sentiment_remote_change'] = data['sentiment_remote_change'].apply(to_list)
# data['sentiment_why_mode'] = data['sentiment_why_mode'].apply(to_list)

In [69]:
# data['sentiment_remote_change'].apply(num_sentiment).value_counts(normalize=True)

In [70]:
# data['sentiment_why_mode'].apply(num_sentiment).value_counts(normalize=True)

In [85]:
count = {'anger': 0, 'fear': 0, 'joy': 0, 'sadness': 0, \
             'analytical': 0, 'confident': 0, 'tentative': 0, 'none': 0}
def count_sentiment(row,args=(count)):
    if type(row) == list and len(row) == 0:
        count['none'] += 1
    if type(row) == list and len(row) != 0:
        for tone in row:
            count[tone['tone_id']] += 1
    return count

In [86]:
def percentage_sentiment(data):
    count_ = pd.Series(data.apply(count_sentiment)[0]).rename('count').reset_index()
    count_['percentage'] = count_['count'] / sum(count_['count'])
    return count_.sort_values('percentage', ascending=False)

In [87]:
data1.head()

,sentiment_remote_change
0,"[{'score': 0.758976, 'tone_id': 'joy', 'tone_n..."
1,"[{'score': 0.579367, 'tone_id': 'analytical', ..."
2,[]
3,"[{'score': 0.916667, 'tone_id': 'sadness', 'to..."
4,"[{'score': 0.620279, 'tone_id': 'analytical', ..."


In [92]:
count = {'anger': 0, 'fear': 0, 'joy': 0, 'sadness': 0, \
             'analytical': 0, 'confident': 0, 'tentative': 0, 'none': 0}
sentiment_remote_change = percentage_sentiment(data1['sentiment_remote_change'].apply(to_list))
count = {'anger': 0, 'fear': 0, 'joy': 0, 'sadness': 0, \
             'analytical': 0, 'confident': 0, 'tentative': 0, 'none': 0}
sentiment_prior_change = percentage_sentiment(data3['sentiment_prior_change'].apply(to_list))
count = {'anger': 0, 'fear': 0, 'joy': 0, 'sadness': 0, \
             'analytical': 0, 'confident': 0, 'tentative': 0, 'none': 0}
sentiment_why_mode = percentage_sentiment(data2['sentiment_why_mode'].apply(to_list))
count = {'anger': 0, 'fear': 0, 'joy': 0, 'sadness': 0, \
             'analytical': 0, 'confident': 0, 'tentative': 0, 'none': 0}
sentiment_why_preference = percentage_sentiment(data4['sentiment_why_preference'].apply(to_list))

In [93]:
print(sentiment_remote_change)

        index  count  percentage
4  analytical   5140    0.344342
6   tentative   2935    0.196624
7        none   2632    0.176325
2         joy   1571    0.105246
3     sadness   1410    0.094460
5   confident    944    0.063241
0       anger    162    0.010853
1        fear    133    0.008910


In [94]:
print(sentiment_prior_change)

        index  count  percentage
4  analytical   3245    0.323433
6   tentative   1964    0.195754
7        none   1914    0.190770
2         joy   1092    0.108841
3     sadness   1083    0.107944
5   confident    460    0.045849
1        fear    172    0.017143
0       anger    103    0.010266


In [95]:
print(sentiment_why_mode)

        index  count  percentage
4  analytical   5714    0.418424
6   tentative   2205    0.161467
2         joy   1925    0.140964
7        none   1830    0.134007
5   confident    939    0.068761
3     sadness    817    0.059827
1        fear    121    0.008861
0       anger    105    0.007689


In [96]:
print(sentiment_why_preference)

        index  count  percentage
4  analytical   6652    0.394450
2         joy   3576    0.212049
6   tentative   2482    0.147177
7        none   1659    0.098375
3     sadness   1314    0.077917
5   confident    925    0.054851
1        fear    177    0.010496
0       anger     79    0.004685
